### Setup

In [ ]:
# install packages
import Pkg;
Pkg.add("CSV");
Pkg.add("DataFrames");
Pkg.add("FreqTables");
Pkg.add("StatsBase");

In [1]:
# load packages
using CSV;
using DataFrames;
using FreqTables;
using StatsBase;

In [2]:
# setup filepaths
path_source = string(@__DIR__,"\\..\\source");
path_dev = string(@__DIR__,"\\..\\dev");
path_output = string(@__DIR__,"\\..\\output");

### Read in dataset

In [3]:
# read in cleaned combined VAERS file
df = CSV.read(joinpath(path_dev,"19-21VAERSCOMB_clean.csv"), DataFrame);
names(df)

8-element Vector{String}:
 "VAERS_ID"
 "RECVDATE"
 "AGE_YRS"
 "VAX_NAME"
 "VAX_TYPE"
 "VAX_MANU"
 "SYMPTOMS"
 "SERIOUS_EVENT"

In [4]:
size(df)

(103050, 8)

In [5]:
first(select(df, ["VAERS_ID", "VAX_NAME", "SYMPTOMS", "SERIOUS_EVENT"]), 10)

,VAERS_ID,VAX_NAME,SYMPTOMS
,Int64,String,String
1,794156,INFLUENZA (SEASONAL) (FLUARIX QUADRIVALENT),"Set([""Injection site joint pain"", ""Injected limb mobility decreased""])"
2,794157,ZOSTER (SHINGRIX),"Set([""Apathy"", ""Injection site pain"", ""Injection site pruritus"", ""Asthenia"", ""Arthralgia"", ""Injection site erythema"", ""Injection site warmth"", ""Injection site swelling"", ""Night sweats"", ""Listless""])"
3,794158,ZOSTER (SHINGRIX),"Set([""Pain"", ""Headache"", ""Nausea"", ""Pyrexia"", ""Chills""])"
4,794160,ZOSTER (SHINGRIX),"Set([""Lip swelling"", ""Lip blister"", ""Pain"", ""Influenza like illness"", ""Asthenia"", ""Injection site erythema"", ""Fatigue"", ""Injection site swelling"", ""Chills""])"
5,794161,ZOSTER (SHINGRIX),"Set([""Pyrexia""])"
6,794163,ZOSTER (SHINGRIX),"Set([""Abdominal pain"", ""Nausea"", ""Pyrexia"", ""Headache"", ""Arthralgia"", ""Fatigue"", ""Dizziness"", ""Myalgia""])"
7,794164,ZOSTER (SHINGRIX),"Set([""Injection site pain""])"
8,794165,INFLUENZA (SEASONAL) (FLUZONE HIGH-DOSE),"Set([""Extra dose administered"", ""No adverse event""])"
9,794165,PNEUMO (PREVNAR13),"Set([""Extra dose administered"", ""No adverse event""])"


In [6]:
# Convert SYMPTOMS strings back to arrays of strings
a = fill([], size(df,1));

for rownumber in 1:size(df, 1)
    str_symptoms = df[rownumber,:SYMPTOMS]
    str_index_start = findfirst(isequal('['), str_symptoms)+2
    str_index_end = findfirst(isequal(']'), str_symptoms)-2
    a[rownumber] = split(df[rownumber,:SYMPTOMS][str_index_start:str_index_end], "\", \"")
end;

df.SYMPTOMS = a;
first(select(df, ["VAERS_ID", "VAX_NAME", "SYMPTOMS", "SERIOUS_EVENT"]), 10)

,VAERS_ID,VAX_NAME,SYMPTOMS
,Int64,String,Array…
1,794156,INFLUENZA (SEASONAL) (FLUARIX QUADRIVALENT),"[""Injection site joint pain"", ""Injected limb mobility decreased""]"
2,794157,ZOSTER (SHINGRIX),"[""Apathy"", ""Injection site pain"", ""Injection site pruritus"", ""Asthenia"", ""Arthralgia"", ""Injection site erythema"", ""Injection site warmth"", ""Injection site swelling"", ""Night sweats"", ""Listless""]"
3,794158,ZOSTER (SHINGRIX),"[""Pain"", ""Headache"", ""Nausea"", ""Pyrexia"", ""Chills""]"
4,794160,ZOSTER (SHINGRIX),"[""Lip swelling"", ""Lip blister"", ""Pain"", ""Influenza like illness"", ""Asthenia"", ""Injection site erythema"", ""Fatigue"", ""Injection site swelling"", ""Chills""]"
5,794161,ZOSTER (SHINGRIX),"[""Pyrexia""]"
6,794163,ZOSTER (SHINGRIX),"[""Abdominal pain"", ""Nausea"", ""Pyrexia"", ""Headache"", ""Arthralgia"", ""Fatigue"", ""Dizziness"", ""Myalgia""]"
7,794164,ZOSTER (SHINGRIX),"[""Injection site pain""]"
8,794165,INFLUENZA (SEASONAL) (FLUZONE HIGH-DOSE),"[""Extra dose administered"", ""No adverse event""]"
9,794165,PNEUMO (PREVNAR13),"[""Extra dose administered"", ""No adverse event""]"


### Exploratory Data Analysis

In [7]:
# Frequency table of vaccines
freq_vax = sort(freqtable(df, :VAX_NAME), rev=true);
freq_vax

112-element Named Vector{Int64}
VAX_NAME                                                │ 
────────────────────────────────────────────────────────┼──────
COVID19 (COVID19 (PFIZER-BIONTECH))                     │ 23630
ZOSTER (SHINGRIX)                                       │ 21666
COVID19 (COVID19 (MODERNA))                             │ 20815
PNEUMO (PNEUMOVAX)                                      │  4810
COVID19 (COVID19 (JANSSEN))                             │  2774
INFLUENZA (SEASONAL) (FLUZONE QUADRIVALENT)             │  2154
INFLUENZA (SEASONAL) (FLUZONE HIGH-DOSE)                │  2044
INFLUENZA (SEASONAL) (FLUCELVAX QUADRIVALENT)           │  1912
INFLUENZA (SEASONAL) (FLUZONE HIGH-DOSE QUADRIVALENT)   │  1819
INFLUENZA (SEASONAL) (AFLURIA QUADRIVALENT)             │  1562
INFLUENZA (SEASONAL) (FLUARIX QUADRIVALENT)             │  1531
PNEUMO (PREVNAR13)                                      │  1408
⋮                                                             ⋮
BCG (NO BRAND

In [8]:
# Create dictionary for VAX_NAME:SYMPTOM
# Step 1: create the dict 
vax_to_symptoms_dict = Dict{String, Set{String}}()
# Step 2: populate the keys (VAERS_ID) of the dict
for rownumber in 1:size(df, 1)
    vax_name = df[rownumber, :VAX_NAME]
    if !haskey(vax_to_symptoms_dict, vax_name)
        # this is the set where we will store all of the symptoms for this VAX_NAME
        vax_to_symptoms_dict[vax_name] = Set{String}()
    end
end
# Step 3: populate the values (SYMPTOMS) of the dict
for rownumber in 1:size(df, 1)
    vax_name = df[rownumber, :VAX_NAME]
    symptoms = df[rownumber, :SYMPTOMS]
    for symptom in symptoms 
        push!(vax_to_symptoms_dict[vax_name], symptom)
    end
end
# View dict
vax_to_symptoms_dict

Dict{String, Set{String}} with 112 entries:
  "PNEUMO (PREVNAR)"        => Set(["Laboratory test", "Pyrexia", "Tenderness",…
  "RABIES (RABIE-VAX)"      => Set(["Blood magnesium normal", "Red blood cell s…
  "INFLUENZA (SEASONAL) (F… => Set(["Sinus headache", "Pregnancy", "Cellulitis"…
  "INFLUENZA (SEASONAL) (F… => Set(["Oropharyngeal pain", "Pyrexia", "Injection…
  "DT ADSORBED (NO BRAND N… => Set(["Burning sensation", "Dry mouth", "Cellulit…
  "YELLOW FEVER (NO BRAND … => Set(["Pain", "Mobility decreased", "Muscle disor…
  "COVID19 (COVID19 (UNKNO… => Set(["Tachypnoea", "Full blood count", "Troponin…
  "ANTHRAX (NO BRAND NAME)" => Set(["Arteriogram coronary normal", "Stevens-Joh…
  "MENINGOCOCCAL B (TRUMEN… => Set(["Abdominal pain lower", "Computerised tomog…
  "HEP A (NO BRAND NAME)"   => Set(["Burning sensation", "Electrocardiogram nor…
  "TD ADSORBED (TDVAX)"     => Set(["Full blood count", "Abdominal pain", "Irri…
  "HEP B (NO BRAND NAME)"   => Set(["Burning sensation", "Product

### Among all 112 vaccines, what are the top reported symptoms:

In [9]:
# Create dict of most reported symptoms for all 112 vaccines
symptoms_all_dupes = reduce(vcat, df.SYMPTOMS)
symptoms_freq_dict = StatsBase.countmap(symptoms_all_dupes)
list_symptoms = unique(reduce(vcat, df.SYMPTOMS))
freq_per_symptom = [symptoms_freq_dict[val] for val in list_symptoms]

# print table
tbl_symptoms_freq = hcat(list_symptoms, freq_per_symptom)
tbl_symptoms_freq[sortperm(tbl_symptoms_freq[:, 2]; rev=true), :]

5565×2 Matrix{Any}:
 "Headache"                                    18880
 "Pyrexia"                                     18160
 "Pain"                                        16730
 "Chills"                                      15906
 "Fatigue"                                     14091
 "Injection site pain"                         13350
 "Pain in extremity"                           12820
 "Nausea"                                      11764
 "Injection site erythema"                     10178
 "Dizziness"                                    9515
 "Myalgia"                                      8116
 "Injection site swelling"                      7972
 "Erythema"                                     7401
 ⋮                                             
 "Therapy cessation"                               1
 "Gastric haemorrhage"                             1
 "Progressive multifocal leukoencephalopathy"      1
 "Nephrostomy"                                     1
 "Nail injury"                 

### Among the 3 COVID-19 vaccines, what are the top reported symptoms:

In [10]:
# Create dict of most reported symptoms for COVID vaccines
df_covid19 = filter(row -> row.VAX_TYPE in ["COVID19"], df)

symptoms_all_dupes = reduce(vcat, df_covid19.SYMPTOMS)
symptoms_freq_dict = StatsBase.countmap(symptoms_all_dupes)
list_symptoms = unique(reduce(vcat, df_covid19.SYMPTOMS))
freq_per_symptom = [symptoms_freq_dict[val] for val in list_symptoms]

# print table
tbl_symptoms_freq = hcat(list_symptoms, freq_per_symptom)
tbl_symptoms_freq[sortperm(tbl_symptoms_freq[:, 2]; rev=true), :]

4347×2 Matrix{Any}:
 "Headache"                            11075
 "Pyrexia"                              8528
 "Chills"                               8365
 "Fatigue"                              7976
 "Pain"                                 7370
 "Nausea"                               6494
 "Dizziness"                            5751
 "Injection site pain"                  4466
 "Pain in extremity"                    4399
 "Myalgia"                              4073
 "Injection site erythema"              3055
 "Arthralgia"                           3015
 "Dyspnoea"                             2936
 ⋮                                     
 "Pancreatitis chronic"                    1
 "Escherichia sepsis"                      1
 "Axillary nerve injury"                   1
 "Nail injury"                             1
 "Vertebral artery stenosis"               1
 "Mini-tracheostomy"                       1
 "Bladder mass"                            1
 "Gallbladder mass"                     

### Contingency tables of Serious Events

In [11]:
# Frequency table of serious events
freq_serious = sort(freqtable(df, :SERIOUS_EVENT), rev=true);
freq_serious

2-element Named Vector{Int64}
SERIOUS_EVENT  │ 
───────────────┼──────
0              │ 92951
1              │ 10099

In [12]:
# Example contingency tables for VAX of interest = "COVID19 (COVID19 (PFIZER-BIONTECH))" and serious report
df.COVID19_PFIZER = (df.VAX_NAME .== "COVID19 (COVID19 (PFIZER-BIONTECH))");
tbl = freqtable(df, :COVID19_PFIZER, :SERIOUS_EVENT)

2×2 Named Matrix{Int64}
COVID19_PFIZER ╲ SERIOUS_EVENT │     0      1
───────────────────────────────┼─────────────
false                          │ 72960   6460
true                           │ 19991   3639

In [13]:
get_PRR(tbl)

LoadError: UndefVarError: get_PRR not defined

In [14]:
df.COVID19_MODERNA = (df.VAX_NAME .== "COVID19 (COVID19 (MODERNA))");
tbl = freqtable(df, :COVID19_MODERNA, :SERIOUS_EVENT)

2×2 Named Matrix{Int64}
COVID19_MODERNA ╲ SERIOUS_EVENT │     0      1
────────────────────────────────┼─────────────
false                           │ 75768   6467
true                            │ 17183   3632

In [15]:
get_PRR(tbl)

LoadError: UndefVarError: get_PRR not defined

In [16]:
df.COVID19_JANSSEN = (df.VAX_NAME .== "COVID19 (COVID19 (JANSSEN))");
tbl = freqtable(df, :COVID19_JANSSEN, :SERIOUS_EVENT)

2×2 Named Matrix{Int64}
COVID19_JANSSEN ╲ SERIOUS_EVENT │     0      1
────────────────────────────────┼─────────────
false                           │ 90332   9944
true                            │  2619    155

In [17]:
get_PRR(tbl)

LoadError: UndefVarError: get_PRR not defined

### Define functions for AIM analysis

In [18]:
function get_freqtable(df, vax_name_str, symptom_str)
    test = select(df, ["VAX_NAME", "SYMPTOMS"])
    # Create dummy variable for vax_name
    test.VAX_IND = (test.VAX_NAME .== vax_name_str)
    # Create dummy variable for symptom
    test.SYMPTOM_IND = zeros(size(test, 1))
    for rownumber in 1:size(test, 1)
        if symptom_str in test[rownumber,:SYMPTOMS]
            test[rownumber,:SYMPTOM_IND] = 1
        end
    end
    
    # Create frequency table
    tbl = freqtable(test, :VAX_IND, :SYMPTOM_IND)
    return tbl
end;

function get_PRR(tbl)
    # Calculate PRR
    a = tbl[2,2]
    b = tbl[2,1]
    c = tbl[1,2]
    d = tbl[1,1]
    PRR = (a/(a+b))/(c/(c+d))
    
    return PRR
end;

In [19]:
get_freqtable(df, "COVID19 (COVID19 (PFIZER-BIONTECH))", "Headache")

2×2 Named Matrix{Int64}
VAX_IND ╲ SYMPTOM_IND │   0.0    1.0
──────────────────────┼─────────────
false                 │ 66105  13315
true                  │ 18065   5565

In [20]:
get_PRR(get_freqtable(df, "COVID19 (COVID19 (PFIZER-BIONTECH))", "Headache"))

1.404721271689326

In [21]:
get_freqtable(df, "COVID19 (COVID19 (MODERNA))", "Headache")

2×2 Named Matrix{Int64}
VAX_IND ╲ SYMPTOM_IND │   0.0    1.0
──────────────────────┼─────────────
false                 │ 67741  14494
true                  │ 16429   4386

In [22]:
get_PRR(get_freqtable(df, "COVID19 (COVID19 (MODERNA))", "Headache"))

1.1955304771966409

### Stratified analysis by Age Group

In [23]:
freqtable(df, :AGE_YRS)

94-element Named Vector{Int64}
AGE_YRS  │ 
─────────┼─────
16.0     │ 1514
17.0     │ 1447
18.0     │ 1153
19.0     │  680
20.0     │  641
21.0     │  670
22.0     │  734
23.0     │  886
24.0     │  894
25.0     │ 1053
26.0     │ 1070
27.0     │ 1162
⋮             ⋮
98.0     │   32
99.0     │   19
100.0    │   17
101.0    │   14
102.0    │   11
103.0    │    6
104.0    │    1
105.0    │    2
106.0    │    2
111.0    │    2
113.0    │    1
115.0    │    4

In [24]:
# Create the following age bins, following RI eligibility groups
# 16 to 39
df.AGE_16to39 = (df.AGE_YRS .>= 16) .& (df.AGE_YRS .<= 39);

# 40 to 49
df.AGE_40to49 = (df.AGE_YRS .>= 40) .& (df.AGE_YRS .<= 49);

# 50 to 59
df.AGE_50to59 = (df.AGE_YRS .>= 50) .& (df.AGE_YRS .<= 59);

# 60 to 64
df.AGE_60to64 = (df.AGE_YRS .>= 60) .& (df.AGE_YRS .<= 64);

# 65 to 74
df.AGE_65to74 = (df.AGE_YRS .>= 65) .& (df.AGE_YRS .<= 74);

# 75+
df.AGE_75ovr = (df.AGE_YRS .>= 75);

In [25]:
# Create PRRs for SERIOUS_EVENT, stratified by COVID vax, stratified by AGE GROUP
AGE_GROUP = ["16 to 39", "40 to 49", "50 to 59", "60 to 64", "65 to 74", "75+", "All (16 to 75+)"];

# Pfizer
COVID19_PFIZER_PRR = zeros(0);
push!(COVID19_PFIZER_PRR, get_PRR(freqtable(df[df.AGE_16to39,:], :COVID19_PFIZER, :SERIOUS_EVENT)));
push!(COVID19_PFIZER_PRR, get_PRR(freqtable(df[df.AGE_40to49,:], :COVID19_PFIZER, :SERIOUS_EVENT)));
push!(COVID19_PFIZER_PRR, get_PRR(freqtable(df[df.AGE_50to59,:], :COVID19_PFIZER, :SERIOUS_EVENT)));
push!(COVID19_PFIZER_PRR, get_PRR(freqtable(df[df.AGE_60to64,:], :COVID19_PFIZER, :SERIOUS_EVENT)));
push!(COVID19_PFIZER_PRR, get_PRR(freqtable(df[df.AGE_65to74,:], :COVID19_PFIZER, :SERIOUS_EVENT)));
push!(COVID19_PFIZER_PRR, get_PRR(freqtable(df[df.AGE_75ovr,:], :COVID19_PFIZER, :SERIOUS_EVENT)));
push!(COVID19_PFIZER_PRR, get_PRR(freqtable(df, :COVID19_PFIZER, :SERIOUS_EVENT)));

# Moderna
COVID19_MODERNA_PRR = zeros(0);
push!(COVID19_MODERNA_PRR, get_PRR(freqtable(df[df.AGE_16to39,:], :COVID19_MODERNA, :SERIOUS_EVENT)));
push!(COVID19_MODERNA_PRR, get_PRR(freqtable(df[df.AGE_40to49,:], :COVID19_MODERNA, :SERIOUS_EVENT)));
push!(COVID19_MODERNA_PRR, get_PRR(freqtable(df[df.AGE_50to59,:], :COVID19_MODERNA, :SERIOUS_EVENT)));
push!(COVID19_MODERNA_PRR, get_PRR(freqtable(df[df.AGE_60to64,:], :COVID19_MODERNA, :SERIOUS_EVENT)));
push!(COVID19_MODERNA_PRR, get_PRR(freqtable(df[df.AGE_65to74,:], :COVID19_MODERNA, :SERIOUS_EVENT)));
push!(COVID19_MODERNA_PRR, get_PRR(freqtable(df[df.AGE_75ovr,:], :COVID19_MODERNA, :SERIOUS_EVENT)));
push!(COVID19_MODERNA_PRR, get_PRR(freqtable(df, :COVID19_MODERNA, :SERIOUS_EVENT)));

# Janssen
COVID19_JANSSEN_PRR = zeros(0);
push!(COVID19_JANSSEN_PRR, get_PRR(freqtable(df[df.AGE_16to39,:], :COVID19_JANSSEN, :SERIOUS_EVENT)));
push!(COVID19_JANSSEN_PRR, get_PRR(freqtable(df[df.AGE_40to49,:], :COVID19_JANSSEN, :SERIOUS_EVENT)));
push!(COVID19_JANSSEN_PRR, get_PRR(freqtable(df[df.AGE_50to59,:], :COVID19_JANSSEN, :SERIOUS_EVENT)));
push!(COVID19_JANSSEN_PRR, get_PRR(freqtable(df[df.AGE_60to64,:], :COVID19_JANSSEN, :SERIOUS_EVENT)));
push!(COVID19_JANSSEN_PRR, get_PRR(freqtable(df[df.AGE_65to74,:], :COVID19_JANSSEN, :SERIOUS_EVENT)));
push!(COVID19_JANSSEN_PRR, get_PRR(freqtable(df[df.AGE_75ovr,:], :COVID19_JANSSEN, :SERIOUS_EVENT)));
push!(COVID19_JANSSEN_PRR, get_PRR(freqtable(df, :COVID19_JANSSEN, :SERIOUS_EVENT)));

tbl_aim4 = hcat(AGE_GROUP, COVID19_PFIZER_PRR, COVID19_MODERNA_PRR, COVID19_JANSSEN_PRR)

7×4 Matrix{Any}:
 "16 to 39"         1.07048  1.05067  0.494113
 "40 to 49"         1.02204  1.1913   0.623983
 "50 to 59"         1.62214  2.15827  0.592879
 "60 to 64"         2.16284  2.68546  0.875149
 "65 to 74"         4.28554  4.49458  0.97147
 "75+"              2.91495  3.07947  1.38491
 "All (16 to 75+)"  1.89328  2.21883  0.563457

In [26]:
df_aim4 = DataFrame(tbl_aim4, Symbol.(["AGE GROUP", "PFIZER-BIONTECH", "MODERNA", "JANSSEN"]))

,AGE GROUP,PFIZER-BIONTECH,MODERNA,JANSSEN
,Any,Any,Any,Any
1,16 to 39,1.07048,1.05067,0.494113
2,40 to 49,1.02204,1.1913,0.623983
3,50 to 59,1.62214,2.15827,0.592879
4,60 to 64,2.16284,2.68546,0.875149
5,65 to 74,4.28554,4.49458,0.97147
6,75+,2.91495,3.07947,1.38491
7,All (16 to 75+),1.89328,2.21883,0.563457


In [27]:
# write to output folder
CSV.write(joinpath(path_output,"aim4.csv"), df_aim4);